In [1]:
from pycomposer.composition_controler import CompositionControler
from pycomposer.midi_vectorlizer import MidiVectorlizer
import numpy as np

# `Builder` in `Builder Patter`.
from pydbm.dbm.builders.rt_rbm_simple_builder import RTRBMSimpleBuilder
# The object of Restricted Boltzmann Machine.
from pydbm.dbm.restricted_boltzmann_machines import RestrictedBoltzmannMachine
# RNN and Contrastive Divergence for function approximation.
from pydbm.approximation.rt_rbm_cd import RTRBMCD
# Logistic Function as activation function.
from pydbm.activation.logistic_function import LogisticFunction
# Softmax Function as activation function.
from pydbm.activation.softmax_function import SoftmaxFunction
from pydbm.activation.relu_function import ReLuFunction
from pydbm.activation.tanh_function import TanhFunction

In [31]:
# `Builder` in `Builder Pattern` for RTRBM.
rtrbm_builder = RTRBMSimpleBuilder()
# Learning rate.
rtrbm_builder.learning_rate = 0.00001
# Set units in visible layer.
rtrbm_builder.visible_neuron_part(
    SoftmaxFunction(), 
    127
)
# Set units in hidden layer.
rtrbm_builder.hidden_neuron_part(
    LogisticFunction(normalize_flag=False, binary_flag=False), 
    10
)
# Set units in RNN layer.
rtrbm_builder.rnn_neuron_part(
    LogisticFunction(normalize_flag=True, binary_flag=False)
)
# Set graph and approximation function.
rtrbm_builder.graph_part(RTRBMCD())
# Building.
rbm = rtrbm_builder.get_result()


In [32]:
composition_controler = CompositionControler(
    rbm=rbm,
    resolution=960,
    initial_tempo=120,
    chord_instrument_num=39,
    melody_instrument_num=0
)

In [33]:
midi_vectorlizer = MidiVectorlizer()
tone_df = midi_vectorlizer.extract("../../parsifal.mid")
tone_df.head(), tone_df.shape

(      program       start         end  pitch  velocity  duration
 6922       44  406.623177  407.560677     51        70   0.93750
 6923       44  406.623177  407.560677     55        70   0.93750
 6063       44  407.091927  407.560677     58        70   0.46875
 6064       44  407.560677  408.498177     56        84   0.93750
 6924       44  407.560677  410.373177     48        73   2.81250, (7252, 6))

In [34]:
composition_controler.learn_pitch(
    tone_df=tone_df, 
    training_count=1, 
    batch_size=200
)

In [35]:
chord_melody_list = composition_controler.create_chord_melody_list(
    octave=4, 
    first_chord="I",
    length=8
)

In [37]:
composition_controler.match_melody_to_chords(
    chord_melody_list=chord_melody_list,
    chord_duration_range=(0.8, 2.4),
    start_time=None,
    #beat=8,
    melody_duration_range=(0.1, 0.3),
    space_range=(0.0, 0.01),
    chord_velocity_range=(70, 90),
    melody_velocity_range=(90, 110)
)

In [38]:
composition_controler.save(file_path="test.mid")